In [ ]:
# rag_test 目的為理解 RAG 原理

# 使用 LangChain
# langchain_community 是 LangChain 2.0 後的新架構，把許多社群貢獻整合獨立出來成工具（如資料庫、向量儲存、API 服務），需額外安裝。
# langchain 是核心套件，負責主要的鏈式應用邏輯，但不再內建所有工具和連接器。

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [27]:
llm = Ollama(model="mistral")

In [ ]:
loader = TextLoader("about.txt")
loader.load()

[Document(metadata={'source': 'about.txt'}, page_content='蘋果公司（英語：Apple Inc.），原稱蘋果電腦公司（英語：Apple Computer, Inc.），是源自美國的跨國科技公司，總部位於美國加州的庫比蒂諾，與亞馬遜、谷歌、微軟、Meta並列為五大科技巨擘。目前的業務包括設計、研發、手機通訊和銷售消費電子、電腦軟體、線上服務和個人電腦。\n\n該公司最著名的硬體產品有iPhone智慧型手機、iPad平板電腦、Mac個人電腦、iPod音樂播放器、Apple Watch智慧型手錶、Apple Vision Pro空間計算電腦、 Apple TV媒體播放器 、AirPods無線耳機和HomePod智慧型喇叭、遊戲機Pippin ；軟體有macOS、iOS、iPadOS、watchOS、tvOS和新的visionOS六大作業系統、iTunes播放器、Safari網頁瀏覽器 、Shazam音樂辨識，還有iLife和iWork創意和生產力套件，以及Final Cut Pro、Logic Pro和Xcode等專業軟體。線上服務有iTunes Store、iOS App Store、Mac App Store、Apple Music、Apple TV+、iMessage和iCloud。蘋果公司還有其他一系列服務包括Apple Store、Genius Bar、AppleCare、Apple Pay、Apple Pay Cash和Apple Card等。\n\n按收入計算，蘋果是全球最大的科技公司，也是全球市值最高的公司之一。2022年1月，它成為了第一家市值達到3兆美元的公司。蘋果公司還擁有很高的品牌忠誠度，並被評為世界最具價值品牌。')]

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separators=[" ", ",", "\n"],
)
text_splitter.split_documents(loader.load())

[Document(metadata={'source': 'about.txt'}, page_content='蘋果公司（英語：Apple'),
 Document(metadata={'source': 'about.txt'}, page_content=' Inc.），原稱蘋果電腦公司（英語：Apple'),
 Document(metadata={'source': 'about.txt'}, page_content='Computer,'),
 Document(metadata={'source': 'about.txt'}, page_content=' Inc.），是源自美國的跨國科技公司，總部位於美國加州的庫比蒂諾，與亞馬遜、谷歌、微軟、Meta並列為五大科技巨擘。目前的業務包括設計、研發、手機通訊和銷售消費電子、電腦軟體、線上服務和個人電腦。'),
 Document(metadata={'source': 'about.txt'}, page_content='\n該公司最著名的硬體產品有iPhone智慧型手機、iPad平板電腦、Mac個人電腦、iPod音樂播放器、Apple'),
 Document(metadata={'source': 'about.txt'}, page_content='Watch智慧型手錶、Apple'),
 Document(metadata={'source': 'about.txt'}, page_content='Vision Pro空間計算電腦、'),
 Document(metadata={'source': 'about.txt'}, page_content='Apple TV媒體播放器'),
 Document(metadata={'source': 'about.txt'}, page_content=' 、AirPods無線耳機和HomePod智慧型喇叭、遊戲機Pippin'),
 Document(metadata={'source': 'about.txt'}, page_content=' ；軟體有macOS、iOS、iPadOS、watchOS、tvOS和新的visionOS六大作業系統、iTunes播放器、Safari網頁瀏覽器'),
 Document(metadata={'s

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
# 實作
# 使用 langchain_community 第三方套件來建立 LLM 物件
llm = Ollama(model="mistral")
# 讀取.txt內容，不用自己手動呼叫 open() 函數讀取檔案
loader = TextLoader("about.txt")

# 把文字分割成小段落
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separators=[" ", ",", "\n"],
)

splited_docs = text_splitter.split_documents(loader.load())

# 改用 nomic-embed-text 模型計算向量(速度較快)
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# 將計算好的embedding存在sqlite db中
vector_db = Chroma.from_documents(
    documents=splited_docs,
    embedding=embeddings,
    persist_directory="db",
    collection_name="about",
)

# 把向量變成檢索器
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

system_prompt = "現在開始使用我提供的情境來回答，只能使用繁體中文，不要有簡體中文字。如果你不確定答案，就說不知道。情境如下:\n\n{context}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "問題: {input}"),
    ]
)


document_chain = create_stuff_documents_chain(llm, prompt_template)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context = []
input_text = input("您想問什麼問題？\n>>> ")

while input_text.lower() != "q":
    response = retrieval_chain.invoke({"input": input_text, "context": context})
    context = response["context"]

    print(response["answer"])